In [20]:
import pandas as pd
from collections import Counter
from nltk.util import ngrams
import nltk

In [63]:
seuss = "<s> I am Sam </s>\
 <s> Sam I am </s>\
 <s> I am Sam </s>\
 <s> I do not like green eggs and ham </s>"

In [64]:
# here is code to create all of the possibilties for the trigrams from the seuss input above
N = 3
trigrams = ngrams(seuss.split(), N)
for grams in trigrams:
  print(grams)

('<s>', 'I', 'am')
('I', 'am', 'Sam')
('am', 'Sam', '</s>')
('Sam', '</s>', '<s>')
('</s>', '<s>', 'Sam')
('<s>', 'Sam', 'I')
('Sam', 'I', 'am')
('I', 'am', '</s>')
('am', '</s>', '<s>')
('</s>', '<s>', 'I')
('<s>', 'I', 'am')
('I', 'am', 'Sam')
('am', 'Sam', '</s>')
('Sam', '</s>', '<s>')
('</s>', '<s>', 'I')
('<s>', 'I', 'do')
('I', 'do', 'not')
('do', 'not', 'like')
('not', 'like', 'green')
('like', 'green', 'eggs')
('green', 'eggs', 'and')
('eggs', 'and', 'ham')
('and', 'ham', '</s>')


In [65]:
seuss_trigrams = nltk.trigrams(seuss)
condition_pairs = (((w0, w1), w2) for w0, w1, w2 in seuss_trigrams)
cfd_seuss = nltk.ConditionalFreqDist(condition_pairs)

In [66]:
def trigram_counts(word_list):
    seuss_trigrams = nltk.trigrams(word_list)
    condition_pairs = (((w0, w1), w2) for w0, w1, w2 in seuss_trigrams)
    fdist = nltk.ConditionalFreqDist(condition_pairs)
    d = Counter()
    for k, v in fdist.items():
        d[k] = v
    return d 

In [67]:
trigram_counts(seuss)

Counter({('<', 's'): FreqDist({'>': 4}),
         ('s', '>'): FreqDist({' ': 7}),
         ('>', ' '): FreqDist({'I': 3, '<': 3, 'S': 1}),
         (' ', 'I'): FreqDist({' ': 4}),
         ('I', ' '): FreqDist({'a': 3, 'd': 1}),
         (' ', 'a'): FreqDist({'m': 3, 'n': 1}),
         ('a', 'm'): FreqDist({' ': 7}),
         ('m', ' '): FreqDist({'<': 4, 'S': 2, 'I': 1}),
         (' ', 'S'): FreqDist({'a': 3}),
         ('S', 'a'): FreqDist({'m': 3}),
         (' ', '<'): FreqDist({'/': 4, 's': 3}),
         ('<', '/'): FreqDist({'s': 4}),
         ('/', 's'): FreqDist({'>': 4}),
         (' ', 'd'): FreqDist({'o': 1}),
         ('d', 'o'): FreqDist({' ': 1}),
         ('o', ' '): FreqDist({'n': 1}),
         (' ', 'n'): FreqDist({'o': 1}),
         ('n', 'o'): FreqDist({'t': 1}),
         ('o', 't'): FreqDist({' ': 1}),
         ('t', ' '): FreqDist({'l': 1}),
         (' ', 'l'): FreqDist({'i': 1}),
         ('l', 'i'): FreqDist({'k': 1}),
         ('i', 'k'): FreqDist({'e': 1}),
 

In [59]:
START_SYMBOL = '*'
STOP_SYMBOL = 'STOP'
MINUS_INFINITY_SENTENCE_LOG_PROB = -1000

def calc_probabilities(words):
    unigram_c = collections.defaultdict(int)
    bigram_c = collections.defaultdict(int)
    trigram_c = collections.defaultdict(int)


    for line in words:
        tokens0 = line.split()
        tokens1 = tokens0 + [STOP_SYMBOL]
        tokens2 = [START_SYMBOL] + tokens0 + [STOP_SYMBOL]
        tokens3 = [START_SYMBOL] + [START_SYMBOL] + tokens0 + [STOP_SYMBOL]
        # unigrams
        for unigram in tokens1:
            unigram_c[unigram] += 1

        # bigrams
        for bigram in nltk.bigrams(tokens2):
            bigram_c[bigram] += 1

    
        # trigrams
        for trigram in nltk.trigrams(tokens3):
            trigram_c[trigram] += 1

    unigrams_len = sum(unigram_c.values())
    unigram_p = {k: math.log(float(v) / unigrams_len, 2) for k, v in unigram_c.items()}

    # calc P(W2|W1) = P(W2,W1) / P(W1) = C(W2,W1) / C(W1)
    unigram_c[START_SYMBOL] = len(words)
    bigram_p = {k: math.log(float(v) / unigram_c[k[0]], 2) for k, v in bigram_c.items()}

    bigram_c[(START_SYMBOL, START_SYMBOL)] = len(words)
    trigram_p = {k: math.log(float(v) / bigram_c[k[:2]], 2) for k, v in trigram_c.items()}
    return trigram_p

In [62]:
trigram_c = collections.defaultdict(int)
seuss.split()

['<s>',
 'I',
 'am',
 'Sam',
 '</s>',
 '<s>',
 'Sam',
 'I',
 'am',
 '</s>',
 '<s>',
 'I',
 'am',
 'Sam',
 '</s>',
 '<s>',
 'I',
 'do',
 'not',
 'like',
 'green',
 'eggs',
 'and',
 'Sam',
 '</s>']

{('*', '*', '<'): -3.5698556083309483,
 ('*', '<', 'STOP'): 0.0,
 ('*', '*', 's'): -3.3999306068886352,
 ('*', 's', 'STOP'): 0.0,
 ('*', '*', '>'): -3.5698556083309483,
 ('*', '>', 'STOP'): 0.0,
 ('*', '*', 'STOP'): -1.9848931076097918,
 ('*', '*', 'I'): -4.569855608330949,
 ('*', 'I', 'STOP'): 0.0,
 ('*', '*', 'a'): -3.5698556083309483,
 ('*', 'a', 'STOP'): 0.0,
 ('*', '*', 'm'): -3.762500686273344,
 ('*', 'm', 'STOP'): 0.0,
 ('*', '*', 'S'): -4.569855608330949,
 ('*', 'S', 'STOP'): 0.0,
 ('*', '*', '/'): -4.569855608330949,
 ('*', '/', 'STOP'): 0.0,
 ('*', '*', 'd'): -5.569855608330948,
 ('*', 'd', 'STOP'): 0.0,
 ('*', '*', 'o'): -5.569855608330948,
 ('*', 'o', 'STOP'): 0.0,
 ('*', '*', 'n'): -4.984893107609792,
 ('*', 'n', 'STOP'): 0.0,
 ('*', '*', 't'): -6.569855608330948,
 ('*', 't', 'STOP'): 0.0,
 ('*', '*', 'l'): -6.569855608330948,
 ('*', 'l', 'STOP'): 0.0,
 ('*', '*', 'i'): -6.569855608330948,
 ('*', 'i', 'STOP'): 0.0,
 ('*', '*', 'k'): -6.569855608330948,
 ('*', 'k', 'STOP'):